In [1]:
import pfb_functions as pfn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import csv

# define case names, the output figure frequency
figure_output_frequency = 24
case_namelist = ['pfclm1_base1','pfclm1_base40','pfclm2_base1','pfclm2_base40']

layer_thicknesslist = [
[2,
0.0456,
0.0228,
0.0228,
0.0171,
0.0057,
0.0182,
0.0137,
0.0137,
0.0137,
0.0137,
0.0137,
0.0114,
0.0091,
0.0046,
0.0023
],
[2,
1.8232,
0.9116,
0.9116,
0.6837,
0.2279,
0.0182,
0.0137,
0.0137,
0.0137,
0.0137,
0.0137,
0.0114,
0.0091,
0.0046,
0.0023,
], #start from here repeat the case 1,40
 [2,
0.0456,
0.0228,
0.0228,
0.0171,
0.0057,
0.0182,
0.0137,
0.0137,
0.0137,
0.0137,
0.0137,
0.0114,
0.0091,
0.0046,
0.0023
],
[2,
1.8232,
0.9116,
0.9116,
0.6837,
0.2279,
0.0182,
0.0137,
0.0137,
0.0137,
0.0137,
0.0137,
0.0114,
0.0091,
0.0046,
0.0023,
]
]

In [2]:
# define output file path
sfpath = '/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/postprocessing/subsurfacestorage_cal'
outputfile_soilname = 'pfclm_soil_storage.csv'
outputfile_saproname = 'pfclm_sapro_storage.csv'

num_case=len(case_namelist)

data_length=336 #168 for pfclm

soilwater_matrix = np.zeros((data_length,num_case),dtype=float)
saprowater_matrix = np.zeros((data_length,num_case),dtype=float)

ncols=198
nrows=198
nlayers=16

In [3]:
def thickness_dom (layer_thickness,ncols,nrows,nlayers):
    thick_dom = np.zeros((ncols,nrows,nlayers),dtype=float)
    for i in range(nlayers):
#         thick_dom[:,:,i] = thick_dom[:,:,i] + layer_thickness[i]*5
        thick_dom[:,:,i] = layer_thickness[i]*5
    return thick_dom

In [4]:
def subsurface_storage (fpath,data_length,num_case,ncols,nrows,case_name,time_step,thick_dom):

#     print (time_step)
    fname_sat = '%s/%s.out.satur.%05d.pfb' %(fpath,case_name, time_step)
    fname_porosity = '%s/%s.out.porosity.pfb' %(fpath,case_name)
    
    satur = pfn.read(fname_sat)
    porosity = pfn.read(fname_porosity)
    
    depth_imcompressive = satur*porosity*thick_dom  #in unit of meter
    # get all soil layers from domain from soil bottom layer and above
    mask_ws_soil  =  depth_imcompressive[:,:,6:]
    mask_ws_soil_volume  =  mask_ws_soil*30*30
    # print (mask_ws_soil)
    mask_ws_sapro =  depth_imcompressive[:,:,1:6]
    mask_ws_sapro_volume =  mask_ws_sapro*30*30
    # print (mask_ws_sapro)
    # sum soil water storage for all cells
    total_soilwater_volume_map = np.sum(mask_ws_soil_volume, axis=2)
    total_saprowater_volume_map = np.sum(mask_ws_sapro_volume, axis=2)

#     total_soilwater_volum_value = np.sum(total_soilwater_vomue_map)
#     total_soilwater_volum_value = np.sum(mask_ws_soil)
    
    total_soilwater_depth_map =np.sum(mask_ws_soil, axis=2)
    total_saprowater_depth_map =np.sum(mask_ws_sapro, axis=2)
    
#     total_soilwater_depth_value =np.mean(total_soilwater_depth_map)
#     total_saprowater_depth_value =np.mean(total_saprowater_depth_map)
    
    return total_soilwater_volume_map,total_saprowater_volume_map,total_soilwater_depth_map,total_saprowater_depth_map

    #                V_compressive = layer_satur*layer_spstorage*layer_press*30*30*thickness_index*5
    #                V_imcompressive = layer_satur*layer_porosity*30*30*thickness_index*5
    #                V_imcompressive = layer_satur*layer_porosity*thickness_index*5
    #                total_V_temp =V_compressive+V_imcompressive
    #                total_V_temp =V_imcompressive
    #                total_V=total_V+total_V_temp
    #                a=total_V.sum()

def plot_ws_2D (time_step,figure_output_frequency,soil_2D_data,sapro_2D_data,sfpath,case_name,variable):
    if time_step%figure_output_frequency==0:
        plt.figure(figsize=(10, 7))
        plt.pcolor(soil_2D_data,cmap='jet')
        plt.colorbar()
        plt.savefig('%s/fig/%s_soilwaterstorage_%s_%05d.png'% (sfpath,case_name,variable,time_step))
        plt.close()     
    
        plt.figure(figsize=(10, 7))
        plt.pcolor(sapro_2D_data,cmap='jet')
        plt.colorbar()
        plt.savefig('%s/fig/%s_saprowaterstorage_%s_%05d.png'% (sfpath,case_name,variable,time_step))
        plt.close()

def write_ws_1D (outputfile_soilname,outputfile_saproname, soil_2D_data,sapro_2D_data, operation):  
    if operation == 'mean':
        a=soil_2D_data.mean()
        b=sapro_2D_data.mean()
        soilwater_matrix[time_step,case_index]=a
        saprowater_matrix[time_step,case_index]=b
        np.savetxt(outputfile_soilname,soilwater_matrix,delimiter=",")
        np.savetxt(outputfile_saproname,saprowater_matrix,delimiter=",")
    else:
        a=np.sum(soil_2D_data)
        b=np.sum(sapro_2D_data)
        soilwater_matrix[time_step,case_index]=a
        saprowater_matrix[time_step,case_index]=b
        np.savetxt(outputfile_soilname,soilwater_matrix,delimiter=",")
        np.savetxt(outputfile_saproname,saprowater_matrix,delimiter=",")


def outlier_remove(data_matrix):
    data_matrix_smooth=data_matrix
    [N_rows, N_col]=data_matrix.shape
    
    for j in range (Y): 
        for i in range (1,X-1):
            diff=(data_matrix[i-1,j]-data_matrix[i,j])/data_matrix[i-1,j]
            if diff>0.2:
                data_matrix_smooth[i,j]=(data_matrix[i-1,j]-data_matrix[i+1,j])/2
    return data_matrix_smooth

def time_series_plot (data_matrix,column_title):
    pd.DataFrame(data=data_matrix, columns=column_title)  # 1st row as the column names
    pd.DataFrame.plot(x='hour', y='pfclm1_base1')


In [5]:
# Main function
if __name__ == "__main__":
    for case_index in range (num_case):
        case_name = case_namelist [case_index]
        print (case_name)
        layer_thickness = layer_thicknesslist [case_index]
        fpath = '/home/chaochen/payette/chaochen/CZO/Chao/Demonstration_example/landlab_Const.Saprolite/extended_simulation/outputs_%s' % (case_name)
        
        for time_step in range (data_length):
            thick_dom=thickness_dom(layer_thickness,ncols,nrows,nlayers)
            (total_soilwater_volume_map,total_saprowater_volume_map,total_soilwater_depth_map,total_saprowater_depth_map)=subsurface_storage (fpath,data_length,num_case,ncols,nrows,case_name,time_step,thick_dom)
            # plot the storage depth, 2D
            plot_ws_2D (time_step,figure_output_frequency,total_soilwater_depth_map,total_saprowater_depth_map,sfpath,case_name,'storageindepth')
            plot_ws_2D (time_step,figure_output_frequency,total_soilwater_volume_map,total_saprowater_volume_map,sfpath,case_name,'storageinvolume')
            if time_step%data_length:
                write_ws_1D (outputfile_soilname,outputfile_saproname,total_soilwater_depth_map,total_saprowater_depth_map,'mean')
                write_ws_1D (outputfile_soilname,outputfile_saproname,total_soilwater_volume_map,total_saprowater_volume_map,'sum')
        

pfclm1_base1
pfclm1_base40
pfclm2_base1
pfclm2_base40
